In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import ElasticNet, ElasticNetCV, LassoCV, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, RobustScaler, scale
import seaborn as sns 

In [5]:
#'atlanta'
#'houston'
#'nyc'
#'cinci'
#'seattle'
place_abbr = 'sf'

tracts_deserts_path = 'data/tract_desert_measures/sf_desert_tracts.csv'

save_data_dir = 'data/tract_health/'
save_data_name =  place_abbr+'_tract_health.csv'
save_transf_data_name =  place_abbr+'_tract_health_logt.csv'

save_regression_path = 'data/tract_health/results/'
save_lasso_name = place_abbr+'_tract_lasso_results'
save_enet_name = place_abbr+'_tract_enet_results'

# Aggregation/Cleaning

In [6]:
tracts_deserts = pd.read_csv(tracts_deserts_path, dtype={'GEOID':'str'})
tracts_deserts = tracts_deserts.rename(columns={'GEOID':'TractFIPS'})

In [9]:
tracts_deserts

,food_closest_travel_times,physical_closest_dist,transport_closest_dist,education_closest_travel_times,worship_closest_travel_times,TractFIPS
0,55.082780,0.024852,0.000259,0.000000,116.293950,06075010100
1,104.510900,0.106618,0.000181,109.609030,63.006050,06075010200
2,97.889350,0.000234,0.210090,16.621742,74.510390,06075010300
3,125.605340,0.046652,0.046652,66.053140,50.311584,06075010400
4,121.480840,0.000211,0.000211,87.040130,159.234540,06075010500
...,...,...,...,...,...,...
192,84.441340,0.285294,0.015445,133.434310,55.752556,06075980401
193,250.735890,0.000360,0.000360,54.229710,145.571400,06075980501
194,280.894650,0.279868,0.372384,159.898420,96.200000,06075980600
195,62.344883,0.118217,0.155635,99.950930,99.950930,06075980900


In [14]:
tracts_health

,StateAbbr,StateDesc,CountyName,CountyFIPS,TractFIPS,TotalPopulation,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,TEETHLOST_Crude95CI,Geolocation,food_closest_travel_times,physical_closest_dist,transport_closest_dist,education_closest_travel_times,worship_closest_travel_times,LowIncomeTracts,PovertyRate,MedianFamilyIncome
0,CA,California,San Francisco,6075,06075017601,7630,11.1,"( 9.6, 12.9)",17.4,"(16.6, 18.1)",...,"( 7.3, 13.9)",POINT (-122.4107073 37.77944763),63.049397,0.000221,0.000221,59.742650,54.111760,1,22.977066,66154.0
1,CA,California,San Francisco,6075,06075016200,2541,7.9,"( 6.7, 9.4)",14.7,"(14.1, 15.3)",...,"( 3.9, 9.0)",POINT (-122.4228953 37.77716215),92.836464,0.157229,0.052362,21.975210,38.325047,0,8.665873,210042.0
2,CA,California,San Francisco,6075,06075030202,4313,6.9,"( 5.7, 8.7)",12.9,"(12.4, 13.5)",...,"( 2.9, 7.8)",POINT (-122.4673281 37.76308855),42.604160,0.282765,0.000159,0.000000,42.724552,0,12.881658,159695.0
3,CA,California,San Francisco,6075,06075016802,3264,8.5,"( 7.1, 10.5)",13.7,"(13.2, 14.4)",...,"( 5.2, 11.1)",POINT (-122.4249484 37.77355128),61.082450,0.052283,0.104761,41.643530,13.327935,0,13.579906,146875.0
4,CA,California,San Francisco,6075,06075025500,8471,14.0,"(11.8, 16.2)",16.8,"(16.2, 17.5)",...,"( 5.9, 10.9)",POINT (-122.4372418 37.7280668),129.039660,0.443646,0.201853,31.619670,31.619670,0,7.553366,116157.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,CA,California,San Francisco,6075,06075017700,2128,10.7,"( 8.9, 13.0)",12.2,"(11.7, 12.7)",...,"( 4.3, 8.6)",POINT (-122.4124937 37.76804375),42.746006,0.339886,0.147534,60.830540,100.488400,0,9.927984,111458.0
191,CA,California,San Francisco,6075,06075025900,4261,12.5,"(10.6, 14.8)",19.3,"(18.6, 20.1)",...,"( 5.8, 12.3)",POINT (-122.4109715 37.72348743),128.438160,0.182158,0.090931,47.924946,39.624947,0,5.406529,125430.0
192,CA,California,San Francisco,6075,06075022903,3384,13.9,"(11.1, 17.0)",13.4,"(12.8, 14.0)",...,"( 4.1, 9.6)",POINT (-122.4062321 37.75175786),67.561650,0.094492,0.000205,28.268590,92.972630,1,5.678233,93359.0
193,CA,California,San Francisco,6075,06075015300,2040,7.2,"( 5.5, 9.6)",15.1,"(14.4, 15.8)",...,"( 3.4, 9.1)",POINT (-122.439345 37.78676373),102.269714,0.178350,0.007977,0.000000,34.317577,0,5.812854,143750.0


In [17]:
tracts_health = pd.read_csv("data/PLACES__Census_Tract_Data__GIS_Friendly_Format___2021_release.csv", dtype={'TractFIPS':'str'})
tracts_health = pd.merge(tracts_deserts, tracts_health, on='TractFIPS', how='left')

income_vars = pd.read_csv('data/tract_incomevars.csv', dtype=str).astype({'PovertyRate':'float', 'TractFIPS':'str'})
tracts_health = pd.merge(tracts_health, income_vars, on='TractFIPS', how='left')

tracts_health.to_csv(save_data_dir+save_data_name, index=False)

In [18]:
desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']

tracts_health_logt = tracts_health.copy()
tracts_health_logt[desert_measures] = tracts_health_logt[desert_measures].apply(lambda x: np.log(x+1))
tracts_health_logt.to_csv(save_data_dir + save_transf_data_name, index=False)

In [27]:
tracts_health[tracts_health.isna().any(axis=1)]

,food_closest_travel_times,physical_closest_dist,transport_closest_dist,education_closest_travel_times,worship_closest_travel_times,TractFIPS,StateAbbr,StateDesc,CountyName,CountyFIPS,...,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,LowIncomeTracts,PovertyRate,MedianFamilyIncome
0,55.082780,0.024852,0.000259,0.00000,116.293950,06075010100,CA,California,San Francisco,6075.0,...,33.7,"(32.5, 35.5)",3.1,"( 2.8, 3.5)",9.8,"( 5.8, 14.5)",POINT (-122.4113772 37.80692331),0,17.324675,NaN
66,17.215357,0.098384,0.000117,146.70750,110.357120,06075017801,CA,California,San Francisco,6075.0,...,32.2,"(31.0, 33.8)",7.0,"( 5.9, 8.1)",16.1,"(10.4, 22.7)",POINT (-122.4011735 37.78202361),1,28.379858,NaN
191,147.663650,0.000120,0.000120,110.70695,102.068440,06075980300,CA,California,San Francisco,6075.0,...,29.0,"(28.2, 30.1)",1.3,"( 1.1, 1.4)",3.1,"( 2.3, 5.2)",POINT (-122.4819131 37.7690297),0,0.000000,NaN
192,84.441340,0.285294,0.015445,133.43431,55.752556,06075980401,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,250.735890,0.000360,0.000360,54.22971,145.571400,06075980501,CA,California,San Francisco,6075.0,...,39.0,"(37.8, 40.1)",7.1,"( 6.3, 8.0)",21.8,"(14.6, 29.0)",POINT (-122.4189916 37.71776415),1,33.482810,NaN
196,84.441340,0.285294,0.015445,133.43431,55.752556,06075990100,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Analysis

In [8]:
name_mapping = {'access2': 'Health insurance access', 'arthritis': 'Arthritis prevalence', 'binge': 'Binge drinking prevalence',
               'bphigh': 'High blood pressure prevalence', 'bpmed': 'Medium blood pressure prevalence', 'cancer': 'Cancer prevalence',
               'casthma': 'Asthma prevalence', 'cervical': 'Cervical cancer screenings', 'chd': 'Coronary heart disease prevalence',
               'checkup': 'Routine checkups', 'cholscreen': 'Cholesterol screenings', 'colon_screen': 'Colon cancer screenings',
               'copd': 'COPD prevalence', 'corem': 'Core men\'s health', 'corew': 'Core women\'s health', 'csmoking': 'Smoking prevalence',
               'dental': 'Dental checkups', 'depression': 'Depression prevalence', 'diabetes': 'Diabetes prevalence', 'ghlth': 'General poor health prevalence',
               'highchol': 'High cholesterol prevalence', 'kidney': 'Chronic kidney disease', 'lpa': 'No physical activity', 'mammouse': 'Mammograms',
               'mhlth': 'Poor mental health prevalence', 'obesity': 'Obesity prevalence', 'phlth': 'Poor physical health', 'sleep': 'Poor sleep prevalence',
               'stroke': 'Stroke prevalence', 'teethlost': 'Teeth loss prevalence'}

results_all_nan = pd.DataFrame({'Health condition': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 'Food': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')],
                       'Physical health': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 'Public transport': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')],
                        'Education': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 'Houses of worship': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')],
                       'Poverty Rate': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 'RSquared': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 
                            'MSE': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')]})
results_nan = pd.DataFrame({'Health condition': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 'Food': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')],
                       'Physical health': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 'Poverty Rate': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 
                            'RSquared': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 
                            'MSE': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')]})
add_vars = ['PovertyRate']

In [9]:
def update_all_vars():
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[0]
        results.iat[i, 2] = regr.coef_[1]
        results.iat[i, 3] = regr.coef_[2]
        results.iat[i, 4] = regr.coef_[3]
        results.iat[i, 5] = regr.coef_[4]
        results.iat[i, 6] = regr.coef_[5]
        results.iat[i, 7] = regr.score(X_test, y_test)
        results.iat[i, 8] = mse_test
def update_vars():
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[0]
        results.iat[i, 2] = regr.coef_[1]
        results.iat[i, 3] = regr.coef_[2]
        results.iat[i, 4] = regr.score(X_test, y_test)
        results.iat[i, 5] = mse_test

## Run Elastic Net Regression:
- For all deserts types; *all_deserts = True*
- Food and physical activity deserts only; *all_deserts = False*
- With log transformed desert measures; *log_transf = True*
- Without log transformed desert measures; *log_transf = False*

In [10]:
#set desert and log transformation params
all_deserts=True
log_transf = False

In [11]:
if all_deserts == True:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']
    results = results_all_nan.copy()
    update_method = update_all_vars
else:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist']
    results = results_nan.copy()
    update_method = update_vars
if log_transf == True:
    df = tracts_health_logt
else:
    df = tracts_health
alphas = []
l1_ratios = []
i=0


for c in df.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        x = df[desert_measures+add_vars].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = df[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        data_centered = center_function(y)

        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

        #l1 ratio is from suggested values in ElasticNetCV documentation
        enet_cv = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], 
                                     cv = 10, normalize=True).fit(X_train,y_train)

        alpha = enet_cv.alpha_
        l1 = enet_cv.l1_ratio_
        alphas.append(alpha)
        l1_ratios.append(l1)

        regr = ElasticNet(alpha=alpha, l1_ratio = l1, normalize=True)  # Could try others, or other parameters?
        regr.fit(X_train, y_train.reshape(-1, 1))

        predictions = regr.predict(X_test)
        y_train_pred = regr.predict(X_train)
        mse_test = mean_squared_error(y_test, predictions)

        if all_deserts==True:
            update_all_vars()
        else:
            update_vars()

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,Poverty Rate,RSquared,MSE
0,Poor mental health prevalence,0.0000,-0.0000,-0.0000,-0.0000,0.000000,1.727414,0.6272,0.894517
1,Core women's health,0.0000,-0.5003,0.0000,0.2367,0.347261,-2.876095,0.6259,6.031242
2,Smoking prevalence,0.0000,0.0000,0.0000,-0.0000,-0.072616,2.212893,0.4618,2.464297
3,Teeth loss prevalence,0.0000,0.0773,0.0000,-0.0000,-0.347655,3.883296,0.4084,5.775399
4,Poor sleep prevalence,0.5421,0.4759,0.2710,-0.1783,-0.532896,2.489687,0.4078,4.331717
5,Dental checkups,-0.3309,-0.9726,-0.3922,0.3213,1.310537,-7.598430,0.3863,33.168579
6,Asthma prevalence,0.0418,-0.0006,0.0000,-0.0000,0.000000,0.631646,0.3359,0.275606
7,Poor physical health,-0.0000,0.0000,0.0134,0.0000,-0.261532,1.988412,0.3320,3.422564
8,Cholesterol screenings,-0.0000,0.0000,0.0812,0.0000,-0.000000,-1.530136,0.3205,4.607766
9,Colon cancer screenings,-0.4498,-0.8945,-0.2410,0.3954,0.899317,-4.312133,0.3148,17.117936


## Run Elastic Net for Whole Dataset

In [12]:
if all_deserts == True:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']
    results = results_all_nan.copy()
    update_method = update_all_vars
else:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist']
    results = results_nan.copy()
    update_method = update_vars
if log_transf == True:
    df = tracts_health_logt
else:
    df = tracts_health
alphas = []
l1_ratios = []
i=0


for c in df.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        x = df[desert_measures+add_vars].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = df[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        data_centered = center_function(y)

        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)

        #l1 ratio is from suggested values in ElasticNetCV documentation
        enet_cv = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], 
                                     cv = 10, normalize=True).fit(X_train,y_train)

        alpha = enet_cv.alpha_
        l1 = enet_cv.l1_ratio_
        alphas.append(alpha)
        l1_ratios.append(l1)

        regr = ElasticNet(alpha=alpha, l1_ratio = l1, normalize=True)  # Could try others, or other parameters?
        regr.fit(xscale, y.reshape(-1, 1))

        predictions = regr.predict(xscale)
        mse_test = mean_squared_error(y, predictions)

        if all_deserts==True:
            update_all_vars()
        else:
            update_vars()

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,Poverty Rate,RSquared,MSE
0,Core women's health,-0.0974,-0.2492,-0.0000,0.1487,0.453948,-2.974776,0.6698,5.685399
1,Poor mental health prevalence,0.0000,-0.0000,-0.0000,-0.0000,0.000000,1.649952,0.6400,1.341761
2,Dental checkups,-0.9677,-0.7671,0.1566,0.1854,1.110621,-7.122031,0.4860,29.421585
3,Smoking prevalence,0.0080,0.0000,0.0000,0.0000,-0.000000,2.105179,0.4782,3.222325
4,Teeth loss prevalence,0.0287,0.0000,-0.0000,-0.0000,-0.093626,3.657190,0.4574,7.277881
5,Poor sleep prevalence,0.6571,0.4431,0.1319,-0.0998,-0.503257,2.325294,0.4484,4.566743
6,Core men's health,-0.4360,-0.4439,0.0270,0.1982,0.650168,-3.528485,0.3979,7.879761
7,Asthma prevalence,0.0000,-0.0087,0.0000,0.0000,0.000000,0.597866,0.3862,0.453656
8,Colon cancer screenings,-0.7720,-0.7503,0.0270,0.2335,0.820805,-4.026016,0.3785,16.644900
9,Poor physical health,0.0000,0.0000,0.0000,0.0000,-0.049902,1.893269,0.3753,3.572646


### Save Output:

In [13]:
if all_deserts == True:
    save_enet_name += '_all'
if log_transf == True:
    save_enet_name += '_logt'
results_round.to_csv(save_regression_path+save_enet_name+'.csv', index=False)

## Run LASSO regression:
- For all deserts types; *all_deserts = True*
- Food and physical activity deserts only; *all_deserts = False*
- With log transformed desert measures; *log_transf = True*
- Without log transformed desert measures; *log_transf = False*

In [14]:
#set desert and log transformation params
all_deserts=True
log_transf = False

In [15]:
if all_deserts == True:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']
    results = results_all_nan.copy()
    update_method = update_all_vars
else:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist']
    results = results_nan.copy()
    update_method = update_vars
if log_transf == True:
    df = tracts_health_logt
else:
    df = tracts_health
alphas = []
l1_ratios = []
i=0


for c in df.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = df[desert_measures+add_vars].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = df[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(X_train, y_train.reshape(-1, 1))
        
        predictions = regr.predict(X_test)
        y_train_pred = regr.predict(X_train)
        mse_test = mean_squared_error(y_test, predictions)
        
        if all_deserts==True:
            update_all_vars()
        else:
            update_vars()

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Public transport,Education,Houses of worship,Poverty Rate,RSquared,MSE
0,Core women's health,0.0000,-0.3654,0.0000,0.1240,2.346587e-01,-2.677055,0.6277,6.212947
1,Poor mental health prevalence,0.0000,-0.0000,-0.0000,-0.0000,0.000000e+00,1.734202,0.6174,0.938008
2,Smoking prevalence,0.0000,0.0563,0.0000,0.0000,-2.195525e-01,2.322609,0.4812,2.586304
3,Dental checkups,0.0000,-0.8010,-0.0000,0.0000,1.189480e+00,-7.456357,0.4287,33.825107
4,Teeth loss prevalence,-0.0000,0.0824,0.0000,-0.0000,-4.305711e-01,3.914670,0.4223,6.106799
5,Poor sleep prevalence,0.1275,0.4590,0.0000,-0.0000,-3.451006e-01,2.415132,0.3754,5.114572
6,Core men's health,0.0000,-0.5468,-0.0000,0.0869,6.650144e-01,-3.733166,0.3571,9.770296
7,Cholesterol screenings,-0.0000,0.0000,0.1409,0.0000,-0.000000e+00,-1.444751,0.3411,4.409439
8,Poor physical health,-0.0303,0.0000,0.0000,0.0000,-2.566057e-01,2.017658,0.3409,3.160424
9,Colon cancer screenings,-0.0000,-0.8590,0.0000,0.1112,8.017364e-01,-4.247120,0.3276,18.347345


## LASSO on Whole Dataset

In [16]:
#set desert and log transformation params
all_deserts=False
log_transf = False

In [17]:
if all_deserts == True:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']
    results = results_all_nan.copy()
    update_method = update_all_vars
else:
    desert_measures = ['food_closest_travel_times', 'physical_closest_dist']
    results = results_nan.copy()
    update_method = update_vars
if log_transf == True:
    df = tracts_health_logt
else:
    df = tracts_health
alphas = []
l1_ratios = []
i=0


for c in df.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = df[desert_measures+add_vars].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = df[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(xscale, y.reshape(-1, 1))
        
        predictions = regr.predict(xscale)
        mse_test = mean_squared_error(y, predictions)
        
        if all_deserts==True:
            update_all_vars()
        else:
            update_vars()

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

,Health condition,Food,Physical health,Poverty Rate,RSquared,MSE
0,Core women's health,0.0000,-0.3667,-2.980269,0.6388,6.061632
1,Poor mental health prevalence,0.0801,-0.0327,1.851053,0.6126,1.286143
2,Smoking prevalence,0.1681,0.0974,2.287026,0.5589,3.119982
3,Dental checkups,-0.2008,-0.7618,-7.303115,0.5141,30.836161
4,Teeth loss prevalence,0.0461,0.0428,3.717824,0.5000,7.283059
5,Poor sleep prevalence,0.3536,0.4299,2.301966,0.4702,4.924632
6,Core men's health,-0.0984,-0.6102,-3.731435,0.4429,8.362901
7,Colon cancer screenings,-0.2393,-0.8034,-4.141164,0.4262,17.495523
8,Poor physical health,0.0000,0.0000,1.970185,0.4040,3.566812
9,General poor health prevalence,0.0099,0.1141,4.037696,0.3713,17.301956


### Save Output:

In [18]:
if all_deserts == True:
    save_lasso_name += '_all'
if log_transf == True:
    save_lasso_name += '_logt'
results_round.to_csv(save_regression_path+save_lasso_name+'.csv', index=False)